# Market Basket Analysis

Market basket analysis scrutinizes the products customers tend to buy together, and uses the information to decide which products should be cross-sold or promoted together. The term arises from the shopping carts supermarket shoppers fill up during a shopping trip.

Association Rule Mining is used when we want to find an association between different objects in a set, find frequent patterns in a transaction database, relational databases or any other information repository.

The most common approach to find these patterns is Market Basket Analysis, which is a key technique used by large retailers like Amazon, Flipkart, etc to analyze customer buying habits by finding associations between the different items that customers place in their “shopping baskets”. The discovery of these associations can help retailers develop marketing strategies by gaining insight into which items are frequently purchased together by customers. The strategies may include:

- Changing the store layout according to trends
- Customers behavior analysis
- Catalog Design
- Cross marketing on online stores
- Customized emails with add-on sales, etc.

### Matrices

- **Support** : Its the default popularity of an item. In mathematical terms, the support of item A is the ratio of transactions involving A to the total number of transactions.


- **Confidence** : Likelihood that customer who bought both A and B. It is the ratio of the number of transactions involving both A and B and the number of transactions involving B.
     - Confidence(A => B) = Support(A, B)/Support(A)


- **Lift** : Increase in the sale of A when you sell B.
    
    - Lift(A => B) = Confidence(A, B)/Support(B)
        
    - Lift (A => B) = 1 means that there is no correlation within the itemset.
    - Lift (A => B) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, A, and B, are more likely to be bought together.
    - Lift (A => B) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, A, and B, are unlikely to be bought together.

**Apriori Algorithm:** Apriori algorithm assumes that any subset of a frequent itemset must be frequent. Its the algorithm behind Market Basket Analysis. Say, a transaction containing {Grapes, Apple, Mango} also contains {Grapes, Mango}. So, according to the principle of Apriori, if {Grapes, Apple, Mango} is frequent, then {Grapes, Mango} must also be frequent.

In [58]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


### Data

In [59]:
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')

In [60]:
order_products = pd.concat([order_products_prior, order_products_train], axis=0, ignore_index=True)
order_products.shape

(33819106, 4)

In [61]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [62]:
order_products.product_id.nunique()

49685

Out of 49685 keeping top 100 most frequent products.

In [63]:
product_counts = order_products.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop = True)
product_counts = product_counts.merge(products, on = 'product_id', how = 'left')
product_counts.head(10)

,product_id,frequency,product_name,aisle_id,department_id
0,24852,491291,Banana,24,4
1,13176,394930,Bag of Organic Bananas,24,4
2,21137,275577,Organic Strawberries,24,4
3,21903,251705,Organic Baby Spinach,123,4
4,47209,220877,Organic Hass Avocado,24,4
5,47766,184224,Organic Avocado,24,4
6,47626,160792,Large Lemon,24,4
7,16797,149445,Strawberries,24,4
8,26209,146660,Limes,24,4
9,27845,142813,Organic Whole Milk,84,16


Keeping 100 most frequent items in order_products dataframe

In [64]:
freq_products = list(product_counts.product_id)
freq_products[1:10]

[13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845]

In [65]:
len(freq_products)

100

In [66]:
order_products = order_products[order_products.product_id.isin(freq_products)]
order_products.shape

(7795471, 4)

In [67]:
order_products.order_id.nunique()

2444982

In [68]:
order_products = order_products.merge(products, on = 'product_id', how='left')
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,28985,2,1,Michigan Organic Kale,83,4
1,2,17794,6,1,Carrots,83,4
2,3,24838,2,1,Unsweetened Almondmilk,91,16
3,3,21903,4,1,Organic Baby Spinach,123,4
4,3,46667,6,1,Organic Ginger Root,83,4


Structuring the data for feeding in the algorithm

In [69]:
basket = order_products.groupby(['order_id', 'product_name'])['reordered'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()


product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
del product_counts, products, order_products, order_products_prior, order_products_train

encoding the units

In [71]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 
    
basket = basket.map(encode_units)
basket.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
basket.size

244498200

In [73]:
basket.shape


(2444982, 100)

Creating frequent sets and rules

In [74]:
reduced_basket = basket.sample(frac=0.2, random_state=42)  # `random_state` ensures reproducibility

# Check the size of the reduced DataFrame
print(f"Original size: {len(basket)} rows")
print(f"Reduced size: {len(reduced_basket)} rows")

Original size: 2444982 rows
Reduced size: 488996 rows


In [75]:
frequent_items = apriori(reduced_basket, min_support=0.01, use_colnames=True, low_memory=True)
frequent_items.head()

d:\sandeepsir\myenv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.015967,(100% Raw Coconut Water)
1,0.025802,(100% Whole Wheat Bread)
2,0.015961,(2% Reduced Fat Milk)
3,0.036109,(Apple Honeycrisp Organic)
4,0.029060,(Asparagus)


In [76]:
frequent_items.tail()

,support,itemsets
126,0.010370,"(Organic Strawberries, Organic Blueberries)"
127,0.010910,"(Organic Hass Avocado, Organic Raspberries)"
128,0.017775,"(Organic Strawberries, Organic Hass Avocado)"
129,0.014677,"(Organic Strawberries, Organic Raspberries)"
130,0.010170,"(Organic Strawberries, Organic Whole Milk)"


In [77]:
frequent_items.shape

(131, 2)

In [78]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1,num_itemsets=10)
rules.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
39,(Limes),(Large Lemon),0.060679,0.065968,0.011980,0.197425,2.992750,1.0,0.007977,1.163795,0.708873,0.104473,0.140742,0.189512
38,(Large Lemon),(Limes),0.065968,0.060679,0.011980,0.181598,2.992750,1.0,0.007977,1.147750,0.712887,0.104473,0.128730,0.189512
56,(Organic Strawberries),(Organic Raspberries),0.113179,0.058340,0.014677,0.129680,2.222831,1.0,0.008074,1.081970,0.620332,0.093578,0.075760,0.190629
57,(Organic Raspberries),(Organic Strawberries),0.058340,0.113179,0.014677,0.251577,2.222831,1.0,0.008074,1.184920,0.584206,0.093578,0.156061,0.190629
50,(Organic Strawberries),(Organic Blueberries),0.113179,0.042878,0.010370,0.091627,2.136939,1.0,0.005517,1.053667,0.599942,0.071182,0.050933,0.166742
51,(Organic Blueberries),(Organic Strawberries),0.042878,0.113179,0.010370,0.241856,2.136939,1.0,0.005517,1.169727,0.555876,0.071182,0.145100,0.166742
41,(Organic Avocado),(Large Lemon),0.075720,0.065968,0.010397,0.137305,2.081397,1.0,0.005402,1.082691,0.562117,0.079189,0.076376,0.147455
40,(Large Lemon),(Organic Avocado),0.065968,0.075720,0.010397,0.157604,2.081397,1.0,0.005402,1.097204,0.556248,0.079189,0.088592,0.147455
53,(Organic Raspberries),(Organic Hass Avocado),0.058340,0.090843,0.010910,0.187009,2.058592,1.0,0.005610,1.118287,0.546090,0.078903,0.105775,0.153554
52,(Organic Hass Avocado),(Organic Raspberries),0.090843,0.058340,0.010910,0.120098,2.058592,1.0,0.005610,1.070188,0.565613,0.078903,0.065584,0.153554


In [79]:
rules.shape

(60, 14)